In [7]:
import json

with open('name_table.json') as f:
    config = json.load(f)

table_name = config['table_name']

Импорт дашборда

In [5]:
import requests

SUPERSET_PROTOCOL = "http"
SUPERSET_HOST = "localhost:8088"
SUPERSET_USERNAME = "admin"
SUPERSET_PASSWORD = "admin"
NEW_ZIP_FILE = "C:/Users/idukh/OneDrive/Рабочий стол/ДИСК D/INNOSTAGE/sql_request/dashboard_export_time.zip"
class Importer:
    def __init__(self):
        self.session = requests.Session()
        self.get_superset_access_token()
        self.get_csrf_token()

    def get_superset_access_token(self):
        # URL для получения токена
        endpoint = "/api/v1/security/login"
        url = f"{SUPERSET_PROTOCOL}://{SUPERSET_HOST}{endpoint}"
        
        # Данные для аутентификации
        credentials = {
            "username": SUPERSET_USERNAME,
            "password": SUPERSET_PASSWORD,
            "provider": "db",
            "refresh": True
        }

        # Запрос на получение токена
        response = self.session.post(url, json=credentials)
        
        if response.status_code == 200:
            access_token = response.json().get('access_token')
            #print("Полученный токен:", access_token)
            # Добавляем токен в заголовки сессии
            self.session.headers.update({
                'Authorization': f'Bearer {access_token}'
            })
        else:
            print("Ошибка при получении токена:", response.status_code, response.text)
            raise Exception(f"Ошибка получения токена: {response.status_code}")

    def get_csrf_token(self):
        # URL для получения CSRF-токена
        endpoint = "/api/v1/security/csrf_token/"
        url = f"{SUPERSET_PROTOCOL}://{SUPERSET_HOST}{endpoint}"
        
        # Запрос на получение CSRF-токена
        response = self.session.get(url)
        if response.status_code == 200:
            csrf_token = response.json().get("result")
            #print("Получен CSRF токен:", csrf_token)
            # Добавляем CSRF-токен в заголовки сессии
            self.session.headers.update({
                'X-CSRFToken': csrf_token
            })
        else:
            print("Ошибка при получении CSRF токена:", response.status_code, response.text)
            raise Exception(f"Ошибка получения CSRF токена: {response.status_code}")

    def import_dashboard(self, zip_file_path):
        # URL для импорта дашборда
        endpoint = "/api/v1/dashboard/import/"
        url = f"{SUPERSET_PROTOCOL}://{SUPERSET_HOST}{endpoint}"
        
        # Открываем файл дашборда для загрузки
        with open(zip_file_path, 'rb') as infile:
            # Формируем данные для отправки
            files = {'formData': (zip_file_path, infile, 'application/zip')}
            payload = {
                'overwrite': 'true'
            }

            # Отправляем запрос на импорт дашборда
            response = self.session.post(url, files=files, data=payload)

            if response.status_code == 200:
                print("Дашборд успешно импортирован.")
            else:
                print("Ошибка при импорте дашборда:", response.status_code, response.text)

if __name__ == "__main__":
#   importer = Importer()
#   importer.import_dashboard(NEW_ZIP_FILE)


SyntaxError: incomplete input (2503097417.py, line 82)

Удаление дашборда

In [1]:
import requests
import json
SUPERSET_PROTOCOL = "http"
SUPERSET_HOST = "localhost:8088"
SUPERSET_USERNAME = "admin"
SUPERSET_PASSWORD = "admin"

class ChartFetcher:
    def __init__(self):
        self.session = requests.Session()
        self.get_superset_access_token()
        self.get_csrf_token()

    def get_superset_access_token(self):
        # URL для получения токена
        endpoint = "/api/v1/security/login"
        url = f"{SUPERSET_PROTOCOL}://{SUPERSET_HOST}{endpoint}"
        
        # Данные для аутентификации
        credentials = {
            "username": SUPERSET_USERNAME,
            "password": SUPERSET_PASSWORD,
            "provider": "db",
            "refresh": True
        }

        # Запрос на получение токена
        response = self.session.post(url, json=credentials)
        
        if response.status_code == 200:
            access_token = response.json().get('access_token')
            #print("Полученный токен:", access_token)
            # Добавляем токен в заголовки сессии
            self.session.headers.update({
                'Authorization': f'Bearer {access_token}'
            })
        else:
            print("Ошибка при получении токена:", response.status_code, response.text)
            raise Exception(f"Ошибка получения токена: {response.status_code}")

    def get_csrf_token(self):
        # URL для получения CSRF-токена
        endpoint = "/api/v1/security/csrf_token/"
        url = f"{SUPERSET_PROTOCOL}://{SUPERSET_HOST}{endpoint}"
        
        # Запрос на получение CSRF-токена
        response = self.session.get(url)
        if response.status_code == 200:
            csrf_token = response.json().get("result")
            #print("Получен CSRF токен:", csrf_token)
            # Добавляем CSRF-токен в заголовки сессии
            self.session.headers.update({
                'X-CSRFToken': csrf_token
            })
        else:
            print("Ошибка при получении CSRF токена:", response.status_code, response.text)
            raise Exception(f"Ошибка получения CSRF токена: {response.status_code}")
        
    def fetch_dashbords(self, name_dashbord):
        # URL для получения всех графиков
        endpoint = "/api/v1/dashboard/"
        url = f"{SUPERSET_PROTOCOL}://{SUPERSET_HOST}{endpoint}"
        
        response = self.session.get(url)

        if response.status_code == 200:
            charts = response.json().get('result', [])
            for chart in charts:
                if name_dashbord == chart['dashboard_title']:
                    return chart['id']
        else:
            print("Ошибка при получении графиков:", response.status_code, response.text)
    def fetch_dashboard_details(self, dashboard_id):
        # URL для получения информации о дашборде
        endpoint = f"/api/v1/dashboard/{dashboard_id}"
        url = f"{SUPERSET_PROTOCOL}://{SUPERSET_HOST}{endpoint}"
        
        response = self.session.get(url)

        if response.status_code == 200:
            # Проверим весь JSON-ответ
            dashboard_details = response.json().get('result', {})
            #print("Полученные данные о дашборде:", dashboard_details)  # вывод для отладки

            try:
                # Извлечение информации о графиках из 'position_json' в dashboard_data
                position_json = json.loads(dashboard_details.get('position_json', '{}'))
                
                # Извлечение всех chartId из структуры 'position_json'
                chart_ids = []
                for chart_key, chart_data in position_json.items():
                    if 'meta' in chart_data and 'chartId' in chart_data['meta']:
                        chart_ids.append(chart_data['meta']['chartId'])
                
                #print("Chart IDs:", chart_ids)
                return chart_ids
            except json.JSONDecodeError:
                print("Ошибка в формате JSON данных.")
            except KeyError as e:
                print(f"Отсутствует ключ в данных: {e}")

                
    def fetch_chart_dataset(self, chart_id):
        endpoint = f"/api/v1/chart/{chart_id}"
        url = f"{SUPERSET_PROTOCOL}://{SUPERSET_HOST}{endpoint}"
        
        response = self.session.get(url)
        
        if response.status_code == 200:
            chart_details = response.json()
            
            result = chart_details.get('result', {})
            params = result.get('params')
            
            if params:
                # Парсим параметры
                params_dict = json.loads(params)
                datasource = params_dict.get("datasource")
                
                if datasource:
                    # Извлекаем dataset_id из формата "166__table" или "166__druid"
                    dataset_id = datasource.split("__")[0]
                    
                    # Запрашиваем детали набора данных
                    dataset_url = f"{SUPERSET_PROTOCOL}://{SUPERSET_HOST}/api/v1/dataset/{dataset_id}"
                    dataset_response = self.session.get(dataset_url)
                    
                    if dataset_response.status_code == 200:
                        dataset_details = dataset_response.json().get('result', {})
                        #dataset_name = dataset_details.get("table_name")
                        #print(f"Dataset ID: {dataset_id}, Dataset Name: {dataset_name}")
                        return dataset_id
                    else:
                        print("Не удалось получить данные для Dataset ID:", dataset_id)
                else:
                    print("Datasource не найден в параметрах графика.")
            else:
                print("Параметры графика не найдены.")
        else:
            print("Ошибка при получении данных о графике:", response.status_code, response.text)
    def delete_chart(self, chart_id):
        # URL для удаления графика
        endpoint = f"/api/v1/chart/{chart_id}"
        url = f"{SUPERSET_PROTOCOL}://{SUPERSET_HOST}{endpoint}"
        
        # Выполняем DELETE-запрос
        response = self.session.delete(url)

        if response.status_code == 200:
            print("График успешно удалён.")
        else:
            print("Ошибка при удалении графика:", response.status_code, response.text)
    def delete_dataset(self, dataset_id):
        # URL для удаления набора данных
        endpoint = f"/api/v1/dataset/{dataset_id}"
        url = f"{SUPERSET_PROTOCOL}://{SUPERSET_HOST}{endpoint}"
        
        # Выполняем DELETE-запрос
        response = self.session.delete(url)

        if response.status_code == 200:
            print("Набор данных успешно удалён.")
        else:
            print("Ошибка при удалении набора данных:", response.status_code, response.text)
    def delete_dashboard(self, dashboard_id):
        # URL для удаления дашборда
        endpoint = f"/api/v1/dashboard/{dashboard_id}"
        url = f"{SUPERSET_PROTOCOL}://{SUPERSET_HOST}{endpoint}"
        
        # Выполняем DELETE-запрос
        response = self.session.delete(url)

        if response.status_code == 200:
            print("Дашборд успешно удалён.")
        else:
            print("Ошибка при удалении дашборда:", response.status_code, response.text)

if __name__ == "__main__":
    chart_fetcher = ChartFetcher()
    name_dashbord = 'innstd_time'
    mass_charts = []
    mass_datasets = []
    id_dashbord=chart_fetcher.fetch_dashbords(name_dashbord)
    mass_charts = chart_fetcher.fetch_dashboard_details(id_dashbord)
    for chart in mass_charts:
        mass_datasets.append(chart_fetcher.fetch_chart_dataset(chart))
    for chart, datasets in zip(mass_charts, mass_datasets):
        chart_fetcher.delete_chart(chart)
        chart_fetcher.delete_dataset(datasets)
    chart_fetcher.delete_dashboard(id_dashbord)
    


График успешно удалён.
Набор данных успешно удалён.
График успешно удалён.
Набор данных успешно удалён.
График успешно удалён.
Набор данных успешно удалён.
График успешно удалён.
Набор данных успешно удалён.
График успешно удалён.
Набор данных успешно удалён.
График успешно удалён.
Набор данных успешно удалён.
Дашборд успешно удалён.
